In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Scegli la lunghezza massima della sequenza. Supportiamo automaticamente RoPE Scaling!
dtype = None  # None per rilevamento automatico. Float16 per Tesla T4, V100, Bfloat16 per Ampere+
load_in_4bit = True  # Usa la quantizzazione a 4 bit per ridurre l'uso della memoria. Può essere False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",  # Scegli il modello, ad esempio "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_..." , # usa questo se utilizzi modelli con gated, come meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Ciao, parli italiano? Dammi "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ciao! Sì, posso parlare un po' di italiano con te. Com'è andata il tuo giorno così?<|eot_id|>


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
import pandas as pd
df = pd.read_parquet("hf://datasets/openbmb/UltraInteract_sft/0000_sft.parquet")

In [ ]:
df.head()

,task,dataset,instruction,response,id,parent_id
0,Coding,codecontest,Write Python code to solve the task.\nMike has...,Step 1: This problem asks us to determine if ...,92a7356beaa75391539cb79624bdf85b7e5d36d5e74e46...,535550938ff58bd73cd7fb85be834bb90b7600daf6b6f6...
1,Coding,codecontest,Write Python code to solve the task.\nMike has...,"Step 1: To solve this problem, we need to che...",362855d34e253130e43b413d3ee7fe91a4f07abc40f227...,535550938ff58bd73cd7fb85be834bb90b7600daf6b6f6...
2,Coding,codecontest,Write Python code to solve the task.\nMike has...,"Step 1: To solve this problem, we need to che...",eea6ea941dbbab75a05da657187447350f8401d3821aad...,535550938ff58bd73cd7fb85be834bb90b7600daf6b6f6...
3,Coding,codecontest,Write Python code to solve the task.\nMike has...,"Step 1: To solve this problem, we need to det...",5a4adc96e3b91bcfd9b6ddda7b52c4a03b6ac94fee59dd...,535550938ff58bd73cd7fb85be834bb90b7600daf6b6f6...
4,Coding,codecontest,Write Python code to solve the task.\nMike has...,"Step 1: To solve this problem, we need to det...",8f015420a9fc36b57077c4fea744e2865744e425b17a35...,535550938ff58bd73cd7fb85be834bb90b7600daf6b6f6...


In [ ]:
df.dropna()

In [ ]:
df.head()

In [ ]:
from datasets import Dataset
import pandas as pd

# Aggiungi una nuova colonna "conversations" basata su `instruction` e `response`
df["conversations"] = df.apply(
    lambda x: [
        {"content": x["instruction"], "role": "user"},
        {"content": x["response"], "role": "assistant"}
    ],
    axis=1
)

# Rimuoviamo colonne non necessarie e convertiamo il DataFrame in un Dataset HuggingFace
dataset = Dataset.from_pandas(df.drop(columns=["instruction", "response"]))

In [ ]:
dataset['conversations'][0]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Configurazione del trainer con correzioni
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="conversations",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=None,  # Disabilita multiprocessing per evitare errori
    packing=False,  # Può accelerare il training per sequenze corte, ma disabilitato per sicurezza
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Ridotto per evitare problemi di memoria
        gradient_accumulation_steps=8,  # Compensa il batch size ridotto
        warmup_steps=5,  # Numero di warmup steps
        max_steps=120,  # Numero massimo di step di training
        learning_rate=2e-4,  # Tasso di apprendimento
        fp16=not is_bfloat16_supported(),  # FP16 se non supportato BFloat16
        bf16=is_bfloat16_supported(),  # Abilita BFloat16 se supportato
        logging_steps=1,  # Frequenza di logging
        optim="adamw_8bit",  # Ottimizzatore a 8-bit
        weight_decay=0.01,  # Decadimento del peso
        lr_scheduler_type="linear",  # Scheduler lineare
        seed=3407,  # Random seed
        output_dir="outputs",  # Directory di output
        report_to="none",  # Disabilita logging verso servizi esterni come WandB
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only

# Ensure 'trainer' is defined before this cell execution
# Assuming 'trainer' is defined in a previous cell with SFTTrainer

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 288,579 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.821200
2,0.700500
3,0.710900
4,0.580000
5,0.746700
6,0.541900
7,0.794300
8,0.922200
9,0.666800
10,0.924300


In [ ]:
# Abilitare la modalità di inferenza per il modello
FastLanguageModel.for_inference(model)  # Abilita inferenza più veloce

# Lista per mantenere il contesto della conversazione
conversation_history = []

def chat_with_model(input_message):
    global conversation_history

    # Aggiungi il messaggio dell'utente alla conversazione
    conversation_history.append({"role": "user", "content": input_message})

    # Costruisci il prompt con lo storico
    inputs = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=True,
        add_generation_prompt=True,  # Necessario per generare output
        return_tensors="pt"
    ).to("cuda")  # Sposta i tensori sulla GPU

    # Configurare il TextStreamer per la generazione di testo
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)

    # Generare il testo con il modello
    response = model.generate(
        input_ids=inputs,  # Passa direttamente il tensor inputs
        streamer=text_streamer,
        max_new_tokens=512,  # Numero massimo di token generati
        use_cache=True,      # Abilita la cache per ottimizzare la generazione
        temperature=1.0,     # Parametro per controllare la creatività
        top_p=0.9            # Usa top-p sampling per risposte fluide
    )

    # Decodifica la risposta del modello
    decoded_response = tokenizer.decode(response[0], skip_special_tokens=True)

    # Aggiungi la risposta del modello alla conversazione
    conversation_history.append({"role": "assistant", "content": decoded_response})

    # Restituisci la risposta del modello
    return decoded_response


# Funzione per stampare il risultato di ogni interazione
def interactive_chat():
    chat_with_model("""Immagina di avere una serie di tre lampadine in una stanza. Ogni lampadina è controllata da un interruttore in una stanza adiacente. Puoi entrare nella stanza delle lampadine solo una volta. Come puoi determinare quale interruttore controlla ciascuna lampadina?""")


# Esegui l'interazione
interactive_chat()

In [ ]:
# Passa alla directory del modello
%cd "/content/drive/My Drive/model"

# Clona il repository di llama.cpp
!git clone --recursive https://github.com/ggerganov/llama.cpp

# Compila llama.cpp
%cd llama.cpp
!make clean && make all -j


In [ ]:
model.save_pretrained_gguf("/content/drive/My Drive/model", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Save to 8bit Q8_0
model.save_pretrained_gguf("model", tokenizer,)

In [ ]:
from google.colab import files
files.download('/content/drive/My Drive/model/unsloth.F16.gguf')